# 1. CUDA 설정 확인

In [2]:
!nvidia-smi

Fri Nov  1 06:09:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#cuda 사용 여부 확인
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
use_cuda = torch.cuda.is_available()
print(use_cuda)
if use_cuda:
  print(torch.cuda.get_device_name(0))

cuda:0
True
Tesla T4


In [4]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # CUDA 사용 가능 여부
print(torch.version.cuda)          # 현재 설치된 CUDA 버전

2.5.0+cu121
True
12.1


In [5]:
!pip install supervision
!pip install roboflow
!pip install autodistill
!pip install autodistill_yolonas
!pip install autodistill_grounding_dino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of supervision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of albumentations to determine which version is compatible with other requirements. This could take a whil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 9.9 MB/s eta 0:00:00
  Created wheel for rf-groundingdino: filename=rf_groundingdino-0.1.2-py2.py3-none-any.whl size=88730 sha256=364445c3c44f4c8598329047c3a201d6c5b9005327d099cc445673dc55165abe
  Stored in directory: /root/.cache/pip/wheels/d6/3f/f8/137b4305e941ad2e0b66cba52e914dda7560308eb91e7e5e47
Successfully built rf-groundingdino


# 2. 모델 테스트

In [23]:
#예상 결과 출력해주는 라이브러리

import cv2
import supervision as sv
import os
from autodistill_grounding_dino import GroundingDINO
from autodistill.detection import CaptionOntology
from groundingdino.util.inference import load_model, load_image, predict, annotate

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'

## 2-1. YOLO-NAS

In [ ]:
#from autodistill_yolonas import YOLONAS
#super_gradient 사용불가 문제로 ...

target_model = torch.load("/content/drive/MyDrive/StrollingPasta/colab/yolo_nas_s.pt")

img = cv2.imread("./Data/india.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)

result = target_model.predict(img, conf=0.35)
result.show(box_thickness=2, show_confidence=True)

RuntimeError: CUDA error: an illegal memory access was encountered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## 2-2. Grounding DINO

In [ ]:
# 이미지 호출
img = cv2.imread("./Data/india.jpg")
img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
#annotated_frame = annotate(image_source=img, boxes=boxes, logits=logits, phrases=phrases)

#오토디스틸에선 이렇게 작동됨
base_model = GroundingDINO(ontology=CaptionOntology({"person": "person"}))

predictions = base_model.predict(img)

annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = annotator.annotate(scene=img, detections=predictions)
labels = []
for i in range(len(predictions)) :
  labels.append("person")

annotated_image = label_annotator.annotate(scene=annotated_image, detections=predictions, labels=labels)

sv.plot_image(img)

trying to load grounding dino directly


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)


final text_encoder_type: bert-base-uncased


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
The `device` argument is deprecated and will be removed in v5 of Transformers.
torch.utils.checkpoint: the use_reentrant parameter should be pas

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 2048 n 20906 k 256 mat1_ld 256 mat2_ld 256 result_ld 2048 abcType 0 computeType 68 scaleType 0

# 3. autodistill로 YOLO-NAS 훈련

## 3-1. raw video에서 이미지 추출

In [6]:
from tqdm.notebook import tqdm

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#데이터셋 경로지정
import os
HOME = os.getcwd()
print(HOME)
!mkdir image

VIDEO_DIR_PATH = f"{HOME}/drive/MyDrive/StrollingPasta/colab/data/semitest"
IMAGE_DIR_PATH = f"{HOME}/image"
FRAME_STRIDE = 10

/content


In [19]:
'''
video_paths = sv.list_files_with_extensions(
    directory=VIDEO_DIR_PATH,
    extensions=["mp4"])

TEST_VIDEO_PATHS, TRAIN_VIDEO_PATHS = video_paths[:2], video_paths[2:]

for video_path in tqdm(TRAIN_VIDEO_PATHS):
    video_name = video_path.stem
    image_name_pattern = video_name + "-{:05d}.png"
    with sv.ImageSink(target_dir_path=IMAGE_DIR_PATH, image_name_pattern=image_name_pattern) as sink:
        for image in sv.get_video_frames_generator(source_path=str(video_path), stride=FRAME_STRIDE):
            sink.save_image(image=image)
'''

0it [00:00, ?it/s]

In [30]:
#중간테스트

video_paths = sv.list_files_with_extensions(
    directory=VIDEO_DIR_PATH,
    extensions=["mp4"])

for video_path in tqdm(video_paths):
    video_name = video_path.stem
    image_name_pattern = video_name + "-{:05d}.png"
    with sv.ImageSink(target_dir_path=IMAGE_DIR_PATH, image_name_pattern=image_name_pattern) as sink:
        for image in sv.get_video_frames_generator(source_path=str(video_path), stride=FRAME_STRIDE):
            sink.save_image(image=image)

  0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
import supervision as sv

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["png"])

print('image count:', len(image_paths))

image count: 2081


## 3-2. GroundingDINO로 오토라벨링

In [32]:
#온톨로지 설정... 찾아야 하는거 : 라벨 형식
#https://docs.autodistill.com/quickstart/
from autodistill.detection import CaptionOntology

#분량이 너무 많아서 중간 발표 전에 테스트 불가능
'''
ontology=CaptionOntology({
    "person walking with dog": "개와 산책하는 사람",
    "person riding bicycle": "자전거 타는 사람",
    "person wearing a hat": "모자 쓴 사람",
    "person with umbrella": "우산 쓴 사람",
    "person holding a phone": "전화하는 사람",
    "blue signs": "파란 표지판",
    "yellow signs": "노란 표지판",
    "truck": "트럭",
    "car": "자동차",
    "helicopter": "헬리콥터",
    "blue bus": "파란색 버스",
    "green bus": "녹색 버스",
    "red bus": "빨간색 버스",
    "bird": "새",
    "yellow cat": "노란 고양이",
    "black cat": "검정 고양이",
    "tree": "나무",
    "flower": "꽃",
    "fire hydrant": "소화전",
    "electric scooter": "전동스쿠터",
    "bicycle": "자전거",
    "ladder": "사다리",
    "manhole": "맨홀",
    "traffic Cone": "라바콘"
})
'''

#중간 테스트 용도
ontology=CaptionOntology({
    "helicopter": "헬리콥터",
    "person riding bicycle": "자전거 타는 사람",
    "electric scooter": "전동스쿠터",
    "traffic Cone": "라바콘"
})

In [ ]:
#자동 라벨링, 데이터셋 제작

DATASET_DIR_PATH = f"{HOME}/drive/MyDrive/StrollingPasta/colab/labeled_data"
base_model = GroundingDINO(ontology=ontology)
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".png",
    output_folder=DATASET_DIR_PATH)

trying to load grounding dino directly
downloading dino model weights


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)


final text_encoder_type: bert-base-uncased



The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Labeling /content/image/Copy of 21-00062.png:   0%|          | 0/2081 [00:00<?, ?it/s]The `device` argument is deprecated and will be removed in

## 3-3. YOLO-NAS-s로 결과물 학습

In [25]:
# Step #5: Train a YOLO-NAS Model
from autodistill_yolonas import YOLONAS

# 원래대로라면 모델명 기입해야 함.
target_model = YOLONAS("")

# model = torch.load("/content/drive/MyDrive/StrollingPasta/colab/yolo_nas_s.pt")
# usr/local/lib/python3.10/dist-packages/autodistill_yolonas/yolonas_model.py

target_model.train("./경로", epochs=20)

# run inference on the new model
pred = target_model.predict("./경로", confidence=0.5)

FileNotFoundError: [Errno 2] No such file or directory: './경로/data.yaml'